This notebook can be used to classify jpeg chips stored locally. Chips are selected randomly from the pool. The interface allows the user to go back, e.g., in order to correct a label, or to stop.
The labels are stored in a json file where each entry corresponds to the chip filename followed by the label for that chip.

In [ ]:
from IPython.display import Image, clear_output, display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Specify the classes and some other parameters.

In [ ]:
classes = ['Building', 'No building']              
samples = 10                             # number of samples to collect
save_every = 2                           # save the json every save_every samples collected
filename = 'labels.json'                 # where to store the labels 
display_width = 256                      # width display in pixels

Label!

In [ ]:
import json
import numpy as np
import os

# Create prompt
prompt = 'Provide input: '
for i, cls in enumerate(classes):
    prompt += '{}: {}, '.format(i+1, cls)

prompt += 'b: back, s: stop '
        
ct = 0                  # counter of labels collected so far   
data = []               # data to be dumped in json
counter_to_save = 0     # counter to save_every

# open the file if it already exists
try:
    with open(filename) as f:
        data = json.load(f)
    ct = len(data)
    # if enough data has been collected for this tile, alert the user and stop
    if ct == samples:
        print 'You have collected enough labels.'
        raise KeyboardInterrupt
except IOError:
    pass
    
print '{} chips labeled so far'.format(ct) 
        
chips = os.listdir('.')
went_back = False
inp = None
game_over = False
        
while ct < samples and not game_over:
            
    print 'Counter: ' + str(ct)

    # pick a chip
    if not went_back:
        index = np.random.randint(1, len(chips)+1, 1)[0]
        chip = chips[index]                
    else:
        chip = data[-1].keys()[0] + '.jpg'
        ct -= 1
        data = data[:ct]
        went_back = False

    print chip

    try:
        Image(filename=chip, format='jpg', width=display_width)
    except IOError:
        continue

    while True:

        inp = str(raw_input(prompt))

        # Go to previous chip
        if inp == 'b':
            if ct >= 1:
                print 'b: ' + str(ct)
                went_back = True
                break
            else:
                print('Can not go back!')
                continue
        elif inp == 's':
            print('stopping')
            game_over = True
            break
        elif inp in map(str, range(1, len(classes)+1)):
            label = classes[int(inp)-1] 
        else:
            print('Please provide a valid input')
            continue

        ct += 1
        data.append({chip[:-4]: label})
        counter_to_save += 1
        break

    if counter_to_save == save_every or ct == samples or game_over:
        with open(filename, 'wb') as f:
            json.dump(data, f)
        counter_to_save = 0

    clear_output()
    
print 'Done!'    